# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

import pickle
import warnings
warnings.filterwarnings('ignore') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [34]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response.db')
df = pd.read_sql_table("Disaster_Response", con=engine)
X = df['message']
Y = df.iloc[:, 4:]

In [35]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [37]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [38]:
def tokenize(text):   
    """
    Tokenize and preprocess a text message.
    Args:
    text: str. A text message.
    Returns:
    tokens: list of str. A list of tokens (words) after performing text preprocessing.
    """
    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
     # Convert all text to lowercase
    text = text.lower()
    
    # Tokenize text into words
    tokens = word_tokenize(text)
    
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [41]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:
y_predict = pipeline.predict(X_test)

In [43]:
# Classification report
print("CLASSIFICATION REPORT:")
for i, column in enumerate(y_test.columns):
        print(column)
        print(classification_report(y_test[column], y_predict[:, i]))

CLASSIFICATION REPORT:
related
             precision    recall  f1-score   support

          0       0.64      0.35      0.45      1518
          1       0.82      0.94      0.87      4985
          2       0.44      0.33      0.38        51

avg / total       0.77      0.79      0.77      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5409
          1       0.84      0.40      0.54      1145

avg / total       0.88      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.87      0.79      3836
          1       0.74      0.53      0.61      2718

avg / total       0.73      0.73      0.72      6554

medical_help
             prec

### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
# Get the parameters of the pipeline
params = pipeline.get_params()

# Print the keys of the parameters dictionary
print("PARAMETER KEYS:")
print(list(params.keys()))

PARAMETER KEYS:
['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estim

In [46]:
# specify parameters for grid search 
parameters = {'clf__estimator__n_estimators' : [50, 100],
              'clf__estimator__max_depth': [None, 10, 20],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [47]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__max_depth': [None, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
cv.best_params_

{'clf__estimator__max_depth': None, 'clf__estimator__n_estimators': 100}

In [51]:
cv.best_score_

0.24249821991659037

In [52]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [57]:
best_estimator_model = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [58]:
y_predict = best_estimator_model.predict(X_test)

In [59]:
# Classification report
print("CLASSIFICATION REPORT:")
for i, column in enumerate(y_test.columns):
        print(column)
        print(classification_report(y_test[column], y_predict[:, i]))

CLASSIFICATION REPORT:
related
             precision    recall  f1-score   support

          0       0.74      0.26      0.39      1518
          1       0.81      0.97      0.88      4985
          2       0.47      0.37      0.42        51

avg / total       0.79      0.80      0.76      6554

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      5409
          1       0.89      0.43      0.58      1145

avg / total       0.89      0.89      0.88      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.76      0.88      0.82      3836
          1       0.79      0.62      0.69      2718

avg / total       0.77      0.77      0.77      6554

medical_help
             prec

In [62]:
avg_accuracy = (y_predict == y_test).mean()
avg_accuracy

related                   0.800885
request                   0.891669
offer                     0.995728
aid_related               0.771437
medical_help              0.924168
medical_products          0.948886
search_and_rescue         0.970857
security                  0.980623
military                  0.969332
child_alone               1.000000
water                     0.951327
food                      0.929356
shelter                   0.933476
clothing                  0.986878
money                     0.979097
missing_people            0.988251
refugees                  0.965822
death                     0.959414
other_aid                 0.866036
infrastructure_related    0.933628
transport                 0.953006
buildings                 0.948886
electricity               0.979707
tools                     0.993592
hospitals                 0.989319
shops                     0.994507
aid_centers               0.988557
other_infrastructure      0.954226
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [68]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):
    """
    The TextLengthExtractor is used to extract the length of the text in each message
    as an additional feature for the machine learning model.
    """
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_length = pd.Series(X).apply(len)
        return pd.DataFrame(X_length)

In [69]:
pipeline2 = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('text_length', TextLengthExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [70]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [71]:
y_predict_2 = pipeline2.predict(X_test)

In [72]:
# Classification report
print("CLASSIFICATION REPORT:")
for i, column in enumerate(y_test.columns):
        print(column)
        print(classification_report(y_test[column], y_predict_2[:, i]))

CLASSIFICATION REPORT:
related
             precision    recall  f1-score   support

          0       0.62      0.36      0.45      1518
          1       0.82      0.93      0.87      4985
          2       0.57      0.08      0.14        51

avg / total       0.77      0.79      0.77      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5409
          1       0.84      0.38      0.52      1145

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.73      0.87      0.79      3836
          1       0.74      0.54      0.62      2718

avg / total       0.73      0.73      0.72      6554

medical_help
             prec

In [76]:
avg_accuracy = (y_predict_2 == y_test).mean()
avg_accuracy

related                   0.792798
request                   0.878700
offer                     0.995728
aid_related               0.729783
medical_help              0.923100
medical_products          0.947971
search_and_rescue         0.971315
security                  0.980317
military                  0.968264
child_alone               1.000000
water                     0.952853
food                      0.922948
shelter                   0.924626
clothing                  0.985963
money                     0.979097
missing_people            0.988099
refugees                  0.967348
death                     0.958193
other_aid                 0.865731
infrastructure_related    0.933628
transport                 0.954226
buildings                 0.948886
electricity               0.979402
tools                     0.993592
hospitals                 0.989472
shops                     0.994507
aid_centers               0.988557
other_infrastructure      0.953616
weather_related     

### 9. Export your model as a pickle file

In [84]:
# Save the model as a pickle file
with open('MLclassifier.pkl', 'wb') as f:
    pickle.dump(best_estimator_model, f)

In [85]:
# Load the saved pickle file
with open('MLclassifier.pkl', 'rb') as f:
    best_estimator_model_from_pickle = pickle.load(f)

In [86]:
best_estimator_model_from_pickle

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.